In [ ]:
# Author: Gargi Singh, Assistant Research Scientist, TTI
# Date: 2021-09-01
# Description: This script performs geocoding using the Google Maps API

import numpy as np
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.distance import geodesic
import getpass

In [ ]:
## Enter Google API Key
GoogleAPIKey = getpass.getpass()
geolocator = GoogleV3(GoogleAPIKey)

In [ ]:
## Load Data to be Geocoded
geocode_address = pd.read_csv('dummy-data.csv')

In [ ]:
## Prepare data for geocoding
geocode_address.columns = ['Name', 'Address', 'City', 'State', 'Zip', 'Longitude', 'Latitude']
geocode_address['LocationNames'] = geocode_address.Address + ", " + geocode_address.City +  ", " + geocode_address.State + " " + (geocode_address.Zip).astype(str) + ", USA"
geocode_address_unique =  pd.DataFrame(geocode_address['LocationNames'].drop_duplicates())
geocode_address_unique.columns = ['LocationNames']

In [ ]:
## Geocode
geocode_address_unique['lat'] = geocode_address_unique['LocationNames'].apply(lambda x: geolocator.geocode(x).latitude)
geocode_address_unique['lon'] = geocode_address_unique['LocationNames'].apply(lambda x: geolocator.geocode(x).longitude)

In [ ]:
## Merge geocoded data with original data
geocode_address = pd.merge(geocode_address, geocode_address_unique, on = 'LocationNames', how = 'left')
geocode_address['VendorCo'] = list(zip(geocode_address.Latitude, geocode_address.Longitude))
geocode_address['GoogleCo'] = list(zip(geocode_address.lat, geocode_address.lon))

In [ ]:
## Calculate the distance between Google and Vendor Coordinates
geocode_address['GeodesicDist'] = geocode_address[['VendorCo', 'GoogleCo']].apply(lambda x: geodesic(x.VendorCo, x.GoogleCo).feet, axis=1)

In [ ]:
## Export results
geocode_address.to_csv('results.csv')